<a href="https://colab.research.google.com/github/ZhongyuGuo/M4R/blob/main/Classification_Edery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
import keras
import random
import re

In [2]:
from google.colab import drive
drive.mount("/content/drive")
df=pd.read_csv("/content/drive/My Drive/M4R/Edery1.csv")
df1=pd.read_csv("/content/drive/My Drive/M4R/papers2019_200000.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# List of categories
catlist=['math','cs','quant-ph','hep-ex','hep-th','astro-ph','nucl-ex','nucl-th']

In [4]:
cat1=catlist[0]
cat2=catlist[2]
cat3=catlist[4]

In [5]:
#Read samples from arXiv dataset for correct word to number mapping
d1=df1[df1['categories']==cat1][0:1500].reset_index(drop=True)
d2=df1[df1['categories']==cat2][0:1500].reset_index(drop=True)
d3=df1[df1['categories']==cat3][0:1500].reset_index(drop=True)
text1=d1.append(d2).reset_index(drop=True)
text1=text1.append(d3).reset_index(drop=True)

In [6]:
# Obtain corresponding abstarcts from the author
# For Ariel Edery, all abstreacts are in at least 1 on the 3 categories, so we take all
text=df

In [7]:
RANDOM_STATE = 50
EPOCHS =15
BATCH_SIZE = 256
TRAINING_LENGTH = 50
TRAIN_FRACTION = 0.7
LSTM_CELLS = 64
VERBOSE = 1
SAVE_MODEL = True

In [8]:
def format_text(text):
    """Add spaces around punctuation and remove references to images/citations."""

    # Add spaces around punctuation
    text = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', text)

    # Remove references to figures
    text = re.sub(r'\((\d+)\)', r'', text)


    # Remove meaningless words
    text = re.sub(' the ',' ',text)
    text = re.sub(' of ',' ',text)
    text = re.sub(' and ',' ',text)
    text = re.sub(' a ',' ',text)
    text = re.sub(' in ',' ',text)
    text = re.sub(' to ',' ',text)
    text = re.sub(' with ',' ',text)
    text = re.sub(' for ',' ',text)
    text = re.sub(' by ',' ',text)
    text = re.sub(' on ',' ',text)
    text = re.sub(' as ',' ',text)
    text = re.sub(' an ',' ',text)
    text = re.sub(' at ',' ',text)
    text = re.sub(' we ',' ',text)
    text = re.sub(' is ',' ',text)
    text = re.sub(' this ',' ',text)
    text = re.sub(' are ',' ',text)
    text = re.sub(' which ',' ',text)
    text = re.sub(' be ',' ',text)
    text = re.sub(' it ',' ',text)
    text = re.sub(' that ',' ',text)
    text = re.sub(' from ',' ',text)
    text = re.sub(' can ',' ',text)
    text = re.sub(' these ',' ',text)
    text = re.sub(' our ',' ',text)
    text = re.sub(' has ',' ',text)
    text = re.sub(' have ',' ',text)
    text = re.sub('.We ','.',text)
    text = re.sub('.That ','.',text)
    text = re.sub('.The ','.',text)
    text = re.sub('.From ','.',text)
    text = re.sub('.Our ','.',text)
    text = re.sub('.In ','.',text)
    text = re.sub('.These ','.',text)
    text = re.sub('.This ','.',text)
    text = re.sub(',that ',',',text)

    # Remove double spaces
    text = re.sub(r'\s\s', ' ', text)
    
    return text

In [9]:
formatted = []

# Iterate through all the original abstracts
for a in text['abstracts']:
    formatted.append(format_text(a))

In [10]:
formatted1=[]
# Iterate through all the original abstracts
for a in text1['abstracts']:
    formatted1.append(format_text(a))

In [11]:
def make_sequences(texts1,texts,training_length=50,lower=True,
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    """Turn a set of texts into sequences of integers"""

    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts1)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    # Convert text to sequences of integers
    features = tokenizer.texts_to_sequences(texts)

    categories=[cat1,cat2,cat3]
    # True labels are not important
    labels=np.zeros((len(text),len(categories)))
    return word_idx, idx_word, num_words, word_counts, features,labels


In [12]:
TRAINING_LENGTH = 50
filters = '.,!"#$%&()*+/:<=>@[\\]^_`{|}~\t\n'
word_idx, idx_word, num_words, word_counts,features,labels = make_sequences(
    formatted1,formatted, TRAINING_LENGTH, lower=True, filters=filters)

There are 22110 unique words.


In [13]:
df1

,Unnamed: 0,categories,abstracts,year
0,0,astro-ph,We systematically explore the evolution of t...,2019
1,1,math,Cofibrations are defined in the category of ...,2019
2,2,astro-ph,We explore the effect of an inhomogeneous ma...,2019
3,3,gr-qc,This paper has been removed by arXiv adminis...,2019
4,4,astro-ph,The most massive elliptical galaxies show a ...,2019
...,...,...,...,...
248749,333099,adap-org,Consider the evolution $$ \frac{\pl m_\iy}{\...,2019
248750,333100,nlin,Consider the evolution $$ \frac{\pl m_\iy}{\...,2019
248751,333101,hep-th,Consider the evolution $$ \frac{\pl m_\iy}{\...,2019
248752,333102,solv-int,A general solution to the Complex Monge-Amp\...,2019


In [14]:
#Make all sequences to same length
pad = len(max(features, key=len))
features=[i + [0]*(pad-len(i)) for i in features]
# Reshape to an array
features1=np.array(features)

In [15]:
# Define Training and Test sets
X_test=features1

y_test=labels

In [16]:
# Load in GloVe
glove=np.loadtxt('/content/drive/My Drive/M4R/glove.6B.100d.txt',dtype='str', comments=None)

In [17]:
# Split words from vector representation
vectors = glove[:, 1:].astype('float')
words = glove[:, 0]
del glove

In [18]:
word_lookup = {word: vector for word, vector in zip(words, vectors)}

embedding_matrix = np.zeros((num_words, vectors.shape[1]))

not_found = 0

for i, word in enumerate(word_idx.keys()):
    # Look up the word embedding
    vector = word_lookup.get(word, None)

    # Record in matrix
    if vector is not None:
        embedding_matrix[i + 1, :] = vector
    else:
        not_found += 1

print(f'There were {not_found} words without pre-trained embeddings.')

There were 9126 words without pre-trained embeddings.


In [19]:
import gc
gc.enable()
del vectors
gc.collect()

161

In [20]:
# Normalize and convert nan to 0
embedding_matrix = embedding_matrix / \
    np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))
embedding_matrix = np.nan_to_num(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [21]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam

In [22]:
def make_word_level_model(embedding_matrix,
                          lstm_cells=64,
                          trainable=False,
                          lstm_layers=1,
                          bi_direc=False):
    """Make a word level recurrent neural network with option for pretrained embeddings
       and varying numbers of LSTM cell layers."""

    model = Sequential()

    # Map words to an embedding
    if not trainable:
        model.add(
            Embedding(
                input_dim=num_words,
                output_dim=embedding_matrix.shape[1],
                weights=[embedding_matrix],
                trainable=False,
                mask_zero=True))
        model.add(Masking())
    else:
        model.add(
            Embedding(
                input_dim=num_words,
                output_dim=embedding_matrix.shape[1],
                weights=[embedding_matrix],
                trainable=True))

    # If want to add multiple LSTM layers
    if lstm_layers > 1:
        for i in range(lstm_layers - 1):
            model.add(
                LSTM(
                    lstm_cells,
                    return_sequences=True,
                    dropout=0.1,
                    recurrent_dropout=0.1))

    # Add final LSTM cell layer
    model.add(
        LSTM(
            lstm_cells,
            return_sequences=False,
            dropout=0.1,
            recurrent_dropout=0.1))
    model.add(Dense(128, activation='relu'))
    # Dropout for regularization
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(len(labels[0]), activation='softmax'))

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


model = make_word_level_model(
    embedding_matrix=embedding_matrix,
    lstm_cells=LSTM_CELLS,
    trainable=False,
    lstm_layers=2,
    bi_direc=False)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2211000   
_________________________________________________________________
masking (Masking)            (None, None, 100)         0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          42240     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3

In [23]:
from IPython.display import Image
model_name = 'pre-trained-rnn'
model_dir = '../models/'
# Plot model is no is functioning due to updeates of Keras
#plot_model(model,show_shapes=True)

In [24]:
# Load and evaluate model
def load_and_evaluate(model_name, return_model=False):
    """Load in a trained model and evaluate with log loss and accuracy"""

    model = load_model(f'/content/drive/My Drive/M4R/3cat642layeredery.h5')

    if return_model:
        return model

In [25]:
model = load_and_evaluate(model_name, return_model=True)

In [26]:
prob=model.predict(X_test)
count=[0,0,0]
for i in range(len(prob)):
  if max(prob[i,:])==prob[i,0]:
    count[0]+=1
  elif max(prob[i,:])==prob[i,1]:
    count[1]+=1
  else:
    count[2]+=1
print(count)

[1, 0, 23]


In [27]:
prob

array([[0.03415843, 0.47351778, 0.49232382],
       [0.02133512, 0.04538091, 0.9332839 ],
       [0.035587  , 0.01183971, 0.9525733 ],
       [0.04646597, 0.01496865, 0.9385654 ],
       [0.07370985, 0.03493878, 0.8913514 ],
       [0.08898628, 0.01232543, 0.89868826],
       [0.0960291 , 0.0355242 , 0.8684467 ],
       [0.07719919, 0.0292128 , 0.89358807],
       [0.02901187, 0.01335072, 0.9576374 ],
       [0.2630705 , 0.01111116, 0.72581834],
       [0.02342292, 0.01323372, 0.9633434 ],
       [0.03195693, 0.01069648, 0.9573466 ],
       [0.06419756, 0.05810872, 0.8776938 ],
       [0.06076048, 0.01181464, 0.92742485],
       [0.0274606 , 0.022265  , 0.9502744 ],
       [0.03557672, 0.00946059, 0.9549627 ],
       [0.04899297, 0.04922823, 0.9017788 ],
       [0.03168702, 0.01949646, 0.9488165 ],
       [0.02601572, 0.01552891, 0.9584553 ],
       [0.02736058, 0.03768641, 0.93495303],
       [0.08955088, 0.0705959 , 0.8398532 ],
       [0.3287405 , 0.32160687, 0.34965268],
       [0.